In [ ]:
# Data preprocessing and formatting script for ablation experiments.

import json
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

In [ ]:
path = '../data/data_raw'

In [ ]:
ablations = ['initsingle', 'initNoevi', 'initevi', 'fullNoevi', 'fullevi']
ablations_names = ['Single LLM', 'Multi-agent', 'Multi+Refine', 'Multi+Tools', 'BioDisco']
scores = ['novelty', 'relevance', 'significance', 'verifiability']

In [ ]:
win_matrix = {i: np.zeros((len(ablations), len(ablations))) for i in scores}
tie_matrix = {i: np.zeros((len(ablations), len(ablations))) for i in scores}

for i in range(len(ablations)):
    for j in range(i, len(ablations)):
        if i != j:
            df = pd.read_csv(os.path.join(path, f'{ablations[i]}_{ablations[j]}.csv'), names=scores)
            for _score in scores:
                assert _score in df.columns, f"Score '{_score}' not found in DataFrame columns."
                win_matrix[_score][i, j] = (df[_score] == -1).sum()
                win_matrix[_score][j, i] = (df[_score] == 1).sum()
                tie_matrix[_score][i, j] = (df[_score] == 0).sum()
                tie_matrix[_score][j, i] = (df[_score] == 0).sum()
                assert win_matrix[_score][j, i] + win_matrix[_score][i, j] + tie_matrix[_score][i, j] == len(df)

In [ ]:
win_matrix

In [ ]:
tie_matrix

In [ ]:
# Create a DataFrame to represent the results in the format P1, P2, P1_win, P2_win, Tie
results = []
for _score in scores:
    for i in range(len(ablations)):
        for j in range(i, len(ablations)):
            if i != j:
                results.append({
                    'score': _score,
                    'P1': ablations_names[i],
                    'P2': ablations_names[j],
                    'P1_win': int(win_matrix[_score][i, j]),
                    'P2_win': int(win_matrix[_score][j, i]),
                    'Tie': int(tie_matrix[_score][i, j])
                })
results_df = pd.DataFrame(results)

In [ ]:
results_df

In [ ]:
results_df.to_csv('results_for_BT.csv', index=False)